In [ ]:

import csv
import matplotlib as mlp
import torch.nn as nn
import numpy as np
import torch.optim as optim
import os
import random
import torch
import PIL
import pandas as pd
import cv2
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
import pathlib
import torch.nn.functional as F
import glob
import matplotlib.pyplot as plt
import shutil
import matplotlib.ticker as ticker
import heapq
from collections import defaultdict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=c1c227384c2ed070e14ccd0883a9ccd62d687c72a49080b7d645ef935bb8c21b
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_train = []
Y_train = []
X_val = []
Y_val = []
X_test = []
Y_test = []

In [ ]:

tsv_file = open("/content/drive/MyDrive/Assignment3/aksharantar_sampled/tel/tel_train.csv")
read_tsv = csv.reader(tsv_file)

pad1 = "\t"
space1 = "\n"
for i in read_tsv:   
    Y_train.append(i[1])
    X_train.append(i[0])

X_train = np.array(X_train)

val_tsv_file = open("/content/drive/MyDrive/Assignment3/aksharantar_sampled/tel/tel_valid.csv")
val_read_tsv = csv.reader(val_tsv_file)
pad2 = "\t"
space2 = "\n"
for i in val_read_tsv:
    Y_val.append(i[1])
    X_val.append(i[0])
X_val= np.array(X_val)

test_tsv_file = open("/content/drive/MyDrive/Assignment3/aksharantar_sampled/tel/tel_test.csv")
test_read_tsv = csv.reader(test_tsv_file)
pad3 = "\t"
space3 = "\n"
for i in test_read_tsv:
    Y_test.append(i[1])
    X_test.append(i[0])

X_test = np.array(X_test)
trainsize=len(X_train)
testsize = len(X_test)
validationsize = len(X_val)

print("Number of Training samples:", trainsize)
print("Number of Test Samples:" , testsize)
print("Number of Validation Samples:", validationsize)

Number of Training samples: 51200
Number of Test Samples: 4096
Number of Validation Samples: 4096


In [ ]:
Y_train = np.array(Y_train)
for i in range(Y_train.shape[0]):
    Y_train[i] = pad1 + Y_train[i] + space1

Y_val = np.array(Y_val)
for i in range(Y_val.shape[0]):
    Y_val[i] = pad2 + Y_val[i] + space2
    
Y_test = np.array(Y_test)
for i in range(Y_test.shape[0]):
    Y_test[i] = pad3+ Y_test[i] + space3


In [ ]:
traintargetsize=len(Y_train)
testtargetsize = len(Y_test)
testtargetsize = len(Y_val)
print("TrainSize of Target ",traintargetsize)
print("TrainSize of Target ",testtargetsize)
print("TrainSize of Target ",testtargetsize)









TrainSize of Target  51200
TrainSize of Target  4096
TrainSize of Target  4096


In [ ]:
input_corpus = set()
output_corpus = set()
input_corpus = set(char for word in X_train for char in word if char not in input_corpus)
output_corpus = set(char for word in Y_train for char in word if char not in output_corpus)
input_corpus.add(" ")
output_corpus.add(" ")
input_corpus = sorted(list(input_corpus))
output_corpus = sorted(list(output_corpus))
val_input_corpus = set()
val_output_corpus = set()
max_encoder_seq_length = max([len(txt) for txt in X_train]) + 2
max_decoder_seq_length = max([len(txt) for txt in Y_train])

print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
val_input_corpus = set(char for word in X_val for char in word if char not in val_input_corpus)
val_output_corpus = set(char for word in Y_val for char in word if char not in val_output_corpus)


Max sequence length for inputs: 30
Max sequence length for outputs: 21


In [ ]:
num_encoder_tokens = len(input_corpus)
num_decoder_tokens = len(output_corpus)
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)

Number of unique input tokens: 27
Number of unique output tokens: 65


In [ ]:


input_data = np.zeros((max_encoder_seq_length,len(X_train)), dtype="int64")
target_data = np.zeros((max_decoder_seq_length,len(X_train)), dtype="int64")
input_data_val = np.zeros((max_encoder_seq_length,len(X_val)), dtype="int64")
target_data_val = np.zeros((max_decoder_seq_length,len(X_val)), dtype="int64")

In [ ]:
input_size_encoder = num_encoder_tokens
input_size_decoder = num_decoder_tokens
output_size = num_decoder_tokens


In [ ]:
def create_char_index(corpus):
    char_index = defaultdict(int)
    for i, char in enumerate(corpus):
        if char not in char_index:
            char_index[char] = i
    return dict(char_index)

input_char_index = create_char_index(input_corpus)
output_char_index = create_char_index(output_corpus)


In [ ]:
for i in range(len(X_train)):
    x = X_train[i]
    y = Y_train[i]
    count=0
    size = x.size
    for t, char in enumerate(x):
        input_data[t, i] = input_char_index[char]
    count=count+1 
    input_data[t + 1 :,i] = input_char_index[" "]
    count1=1
    for t, char in enumerate(y):
        target_data[t, i] = output_char_index[char]
        count1=count+1     
    target_data[t + 1 :,i] = output_char_index[" "]
    

data_type = torch.int64
data_type1=torch.float64
for i in range(len(X_val)):
    x=X_val[i]
    y=Y_val[i]
    count = 0
    size - x.size
    for t, char in enumerate(x):
        input_data_val[t, i] = input_char_index[char]
        count = count +1
    input_data_val[t + 1 :,i] = input_char_index[" "]
    
    count = 0
    for t, char in enumerate(y):
        target_data_val[t, i] = output_char_index[char]
        count = count +1
            
    target_data_val[t + 1 :,i] = output_char_index[" "]

input_data_val = torch.tensor(input_data_val,dtype=data_type)
target_data_val = torch.tensor(target_data_val,dtype=data_type)

In [ ]:

input_data = torch.tensor(input_data,dtype=data_type)
reverse_input_char_index = {i: char for char, i in input_char_index.items()}
reverse_target_char_index = {i: char for char, i in output_char_index.items()}
target_data = torch.tensor(target_data,dtype=data_type)

In [ ]:
#LSTM RUN Only
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.2):
        x = target[0]
        variables = 0
        target_vocab_size = num_decoder_tokens
        input_vocab_size = num_encoder_tokens
        outputs = torch.zeros(target.shape[0], source.shape[1], target_vocab_size).to(device)
        
        hidden, cell=self.encoder(source)
        ht = hidden[-1, :, :]
        res = cell[-1,:,:]

        for t in range(1, target.shape[0]):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            
            outputs[t] = output
            res = output[0:]
          
            best_guess = output.argmax(1)
            
            if random.random() < teacher_force_ratio:
                x = target[t]
            else:
                x = best_guess

        return outputs
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout,size):
        super(Encoder, self).__init__()
        self.dropout,self.num_layers,self.bidirectional,self.embedding = nn.Dropout(dropout),num_layers,True,nn.Embedding(input_size, embedding_size)
        self.hidden_size,self.identity_init,self.init_h0 = hidden_size,torch.eye(hidden_size),nn.Linear(num_layers * 2, num_layers)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc_hidden,self.fc_cell = nn.Linear(hidden_size * 2, hidden_size), nn.Linear(hidden_size * 2, hidden_size)     

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        a = self.embedding(x)
        embedding = self.dropout(a)
        identity_init = self.identity_init
        embedded = self.embedding(x).view(1, 1, -1)
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)
        init_h0 = self.init_h0

        return hidden, cell
    def initHidden(self):
        value = 1
        return torch.zeros(value, value, self.hidden_size, device=device)
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout,beam_width):
        super(Decoder, self).__init__()
        self.dropout,self.num_layers,self.bidirectional,self.embedding = nn.Dropout(dropout), num_layers,True,nn.Embedding(input_size, embedding_size)
        self.hidden_size,self.identity_init,self.init_h0 = hidden_size,torch.eye(hidden_size), nn.Linear(num_layers * 2, num_layers)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()

    def forward(self, x, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)
        hid=hidden.squeeze(0)
        embedding = self.dropout(self.embedding(x))
        embed=embedding[0:]
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs).squeeze(0)
        results=predictions[0:]
        results = self.softmax(predictions)


        return predictions, hidden, cell



In [ ]:
def beam_search(model, word, input_char_index, output_char_index, reverse_input_char_index,
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length,
                num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):

    word_t = ''
    word_new = ''
    
    sizeofword = len(word)
    data = np.zeros((max_encoder_seq_length, 1), dtype="int64")
    for i in range(sizeofword):
        data[i, 0] = input_char_index[word[i]]
    
    data[sizeofword:, 0] = input_char_index[" "]
    data = torch.tensor(data, dtype=torch.int64).to(device)

    with torch.no_grad():
        hidden,cell = model.encoder(data)
    padding = '\t'
    value = 1
    ovalue=-1
    initial_sequence = torch.tensor(np.array(output_char_index[padding]).reshape(value,)).to(device)
    zero = 0
    res =hidden.unsqueeze(0)
    beam = [(0.0, initial_sequence, hidden.unsqueeze(0))]  # [(score, sequence, hidden)]
    start ='\n'
    for _ in range(max_decoder_seq_length):
        candidates = []
        options=[]
        for score, seq, hidden in beam:
            count = 0
            last_token = seq[ovalue].item()
            if last_token == output_char_index[start]:
                # If the sequence ends with the end token, add it to the candidates
                candidates.append((score, seq, hidden))
                count =count+1
                continue

            x = torch.tensor(np.array(last_token).reshape(value,)).to(device)
            output, hidden,cell = model.decoder(x, hidden.squeeze(0),cell)
            res = output[0:]
            probabilities = F.softmax(output, dim=1)
            result =probabilities
            # Get the top-k probabilities and tokens
            topk_probs, topk_tokens = torch.topk(probabilities, k=beam_width)
            tops = topk_probs[0]
            topk = topk_tokens[0]
            for prob, token in zip(tops, topk):
                new_s= token.unsqueeze(0)
                new_seq = torch.cat((seq, new_s), dim=0)
                new_hidden = hidden.clone().unsqueeze(0)
                penalty = len(new_seq) - 1
                len_pen = length_penalty
                length_penalty_factor = ((penalty) / 5) **  len_pen   # Adjust penalty factor as needed
                candidates.append((score + torch.log(prob).item() / length_penalty_factor, new_seq, new_hidden))
                options.append((score + torch.log(prob).item() / length_penalty_factor))
        # Select top-k candidates based on the accumulated scores
        beam = heapq.nlargest(beam_width, candidates, key=lambda x: x[0])

    # Select the best sequence from the beam as the output
    best_res = 0
    word_pred=''
    best_score, best_sequence, _ = max(beam, key=lambda x: x[0])
    word_t = ''.join([reverse_target_char_index[val.item()] for val in best_sequence[value:ovalue]])
    word_pred = word_t
    return word_t

In [ ]:

learning_rate = 0.001



In [ ]:
def training(num_encoder_tokens,input_embedding_size, dp, cell_type, hidden_size, num_enc_layers, num_dec_layers,num_epochs,output_size,input_size_decoder,batch_size,beam_width):
    input = []
    correct = []
    decoded = []
    ex = []
    if(cell_type=="LSTM"):
        encoder_net = Encoder(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers,dp,output_size).to(device)
        decoder_net = Decoder(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp,beam_width).to(device)
        criterion = nn.CrossEntropyLoss()
        data_train = torch.split(input_data,batch_size,dim=1)
        target_train = torch.split(target_data,batch_size,dim=1)
        learning_rate=0.001
        total_words = len(X_val)
        model = Seq2Seq(encoder_net, decoder_net).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
 
        for epoch in range(num_epochs):
            correct_output = 0
            print(f"[Epoch {epoch} / {num_epochs}]")
            model.eval()
            model.train()

            train_size = len(data_train)
            for i in range(train_size):
                neg = -1
                pos = 1
                a = data_train[i]
                b = target_train[i]
               # Get input and targets and get to cuda
                inp_data = a.to(device)
                target = b.to(device)
                # Forward prop
                output = model(inp_data, target)
                size = output.shape[2]
                temp = output[pos:]
                temp1 =temp.reshape(neg, size)
                output = temp1
                tar = target[pos:]
                target = tar.reshape(neg)
                optimizer.zero_grad()
                loss = criterion(output, target)
                # Back prop
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
                # Gradient descent step
                optimizer.step()
            model.eval()
            
            correct_prediction = 0
            result_current = 0
            for i in range(total_words):
    
                decoded_sentence = beam_search(model,X_val[i], input_char_index, output_char_index, reverse_input_char_index, 
              reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
              num_encoder_tokens, num_decoder_tokens,1,device)
                true_output = Y_val[i][1:-1]
                if true_output== decoded_sentence:
                     correct_prediction =correct_prediction + 1
            result_current = correct_prediction
            test_accuracy = correct_prediction / total_words

            print(test_accuracy)
            wandb.log({'val_accuracy' : test_accuracy*100})
        total_words = len(X_test)
        correct_prediction = 0 
        result_current = 0
        for i in range(total_words):
            decoded_sentence = beam_search(model,X_test[i], input_char_index, output_char_index, reverse_input_char_index, 
              reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
              num_encoder_tokens, num_decoder_tokens,1,device)
            true_output = Y_test[i][1:-1]
            if true_output == decoded_sentence:
                  correct_prediction = correct_prediction+ 1
        result_current = correct_prediction
        test_accuracy = correct_prediction / total_words
        print("Test Accuracy is :")
        print(test_accuracy)
 
        np.random.seed(10)
        for index in range(total_words):
            decoded_sentence = beam_search(model,X_test[index], input_char_index, output_char_index, reverse_input_char_index, 
              reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
              num_encoder_tokens, num_decoder_tokens,1,device)
            true_output = Y_test[index][pos:neg]
            if true_output == decoded_sentence:
                ex.append("Yes")
            else:
                ex.append("No")        
        
            input.append(X_test[index])
            correct.append(Y_test[index][pos:neg])
            decoded.append(decoded_sentence)
        import pandas as pd

        grid = {'Input Word': input, 'True Output' : correct, 'My Decoded Output' : decoded, "Matching or Not" : ex}

        df=pd.DataFrame(grid)
        #df= pd.DataFrame.from_dict(grid,orient='index')
        df.to_csv('/content/drive/MyDrive/Assignment3/aksharantar_sampled/tel/Van.csv', index=False,header=True)
        print(df.head(20))
        pd.DataFrame(grid)
        #print(pd.DataFrame(grid))

In [ ]:
import wandb
wandb.login(key='17d991db26320e751b170877037d1067a164fe6d')
wandb.init(project="Assignment_3_F")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs22m046 (harshvrma). Use `wandb login --relogin` to force relogin


In [ ]:
training(input_size_encoder ,512, 0.3, "LSTM",512, 3, 3,40,num_decoder_tokens,num_decoder_tokens,512,3)

[Epoch 0 / 40]
0.0
[Epoch 1 / 40]
0.0
[Epoch 2 / 40]
0.0009765625
[Epoch 3 / 40]
0.049072265625
[Epoch 4 / 40]
0.18115234375
[Epoch 5 / 40]
0.292236328125
[Epoch 6 / 40]
0.35888671875
[Epoch 7 / 40]
0.385498046875
[Epoch 8 / 40]
0.42919921875
[Epoch 9 / 40]
0.4541015625
[Epoch 10 / 40]
0.466796875
[Epoch 11 / 40]
0.451171875
[Epoch 12 / 40]
0.489990234375
[Epoch 13 / 40]
0.498046875
[Epoch 14 / 40]
0.49853515625
[Epoch 15 / 40]
0.4990234375
[Epoch 16 / 40]
0.520751953125
[Epoch 17 / 40]
0.53173828125
[Epoch 18 / 40]
0.533203125
[Epoch 19 / 40]
0.531982421875
[Epoch 20 / 40]
0.51513671875
[Epoch 21 / 40]
0.537353515625
[Epoch 22 / 40]
0.5537109375
[Epoch 23 / 40]
0.553466796875
[Epoch 24 / 40]
0.551025390625
[Epoch 25 / 40]
0.564453125
[Epoch 26 / 40]
0.53857421875
[Epoch 27 / 40]
0.5478515625
[Epoch 28 / 40]
0.546142578125
[Epoch 29 / 40]
0.551025390625
[Epoch 30 / 40]
0.550048828125
[Epoch 31 / 40]
0.554931640625
[Epoch 32 / 40]
0.5595703125
[Epoch 33 / 40]
0.54638671875
[Epoch 34 / 4